# World Model Policy Evaluation

This notebook demonstrates how to rollout OpenVLA in the Bridge (WidowX) environment from within our world model, and how to grade it using a VLM.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from PIL import Image
import requests
import mediapy as media
from tqdm import tqdm
import torch
import einops
import base64
import math
import re
import cv2
from openai import OpenAI
import random
from pathlib import Path
import json

## Action scaling

In [4]:
def rescale_bridge_action(a):
    """
    Rescale Bridge (WidowX) action to the ranges expected by the world model.
    We need to call this function on the unnormalized action values returned by the policy.
    """
    wv_lo = -0.05
    wv_hi = +0.05
    wv_post_scale_max = +1.75
    wv_post_scale_min = -1.75
    rd_lo = -0.25
    rd_hi = +0.25
    rd_post_scale_max = +1.4
    rd_post_scale_min = -1.4
    # rescale end effector
    a[:3] = (a[:3] - wv_lo) / (wv_hi - wv_lo) * (
        wv_post_scale_max - wv_post_scale_min
    ) + wv_post_scale_min
    # rescale joint rotations
    a[3:6] = (a[3:6] - rd_lo) / (rd_hi - rd_lo) * (
        rd_post_scale_max - rd_post_scale_min
    ) + rd_post_scale_min
    # threshold the gripper
    a[6] = torch.where(a[6] > 0.8, -1.0, +1.0)
    return a


## VLM evaluation utils

Set your OpenAI key:
```
export OPENAI_API_KEY=<your key here>
```

In [5]:
def encode_video(video, stride=20):
    frames, idx = [], 0
    for idx, frame in enumerate(video):
        if idx % stride == 0:
            if (frame == 0).all():
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            _, buf = cv2.imencode(".jpg", frame)
            frames.append(base64.b64encode(buf).decode())
    return frames


def predict(video, task, n=5):
    frames = encode_video(video)
    prompt = f"""
Here is a sequence of frames from a robot policy which has been rolled out in a video-generation-based world model. I need your help determining whether the policy is successful. How successfully does the robot complete the following task?
Task: {task["instruction"]}
Subtasks: {task["subtasks"]}

For each subtask, give the model a score of 0 (fail) or 1 (success). Explain your reasoning. Finally, output "Total Score: <score>", where <score> is the sum of the scores on the subtasks above.

Note: Since this video was generated by a video prediction model (conditioned on robot actions), it may contain some artifacts due to the video model capacity.
Note: For the final score output, make sure to follow the "Total Score: <score>" format exactly so the score can be parsed out using a regex.
""".strip()
    client = OpenAI()
    messages = [
        {
            "role": "user",
            "content": [prompt, *[{"image": f, "resize": 256} for f in frames]],
        }
    ]
    print(prompt)
    res = client.chat.completions.create(model="gpt-4o", messages=messages, n=n)
    total_score = 0
    n_scores = 0
    for c in res.choices:
        print(c.message.content)
        m = re.search(r"Total Score: (\d+)", c.message.content.replace("**", ""))
        if m:
            total_score += (
                int(m.group(1)) / 2.0
            )  # divide by 2 since there are 2 subtasks
            n_scores += 1
    avg_score = total_score / n_scores
    print(f"Average Score: {avg_score}")
    return avg_score

## Policy Evaluation Loop

In [21]:
def evaluate_policy(wm, vla, tasks, rollout_length=40):
    """
    Rollout an OpenVLA model on a list of tasks, and return the score on each task.
    Arguments:
        wm: WorldModel
        vla: An OpenVLA model from `transformers`
        tasks: A list of N tasks in loaded from a json. See "put_carrot_on_plate.json" for an example of the format.
    Returns:
        scores: A list of N scores from the VLM corresponding to each input task.
    """
    scores = []
    for task_i in tqdm(tasks, desc="completing tasks"):
        start_frame = np.array(
            Image.open(requests.get(task_i["im_0_path"], stream=True).raw).resize(
                (256, 256)
            )
        )
        media.show_image(start_frame)
        wm.reset(torch.from_numpy(start_frame).cuda().float() / 255.0)

        frames = [start_frame]
        for step in tqdm(range(rollout_length)):
            curr_frame = Image.fromarray(frames[-1])
            prompt = f"In: What action should the robot take to {task_i['instruction']}?\nOut:"
            inputs = processor(prompt, curr_frame).to(
                device="cuda", dtype=torch.bfloat16
            )
            actions = vla.predict_action(
                **inputs, unnorm_key="bridge_orig", do_sample=False
            )

            a = torch.tensor(actions)[0].cuda()
            # NOTE: OpenVLA outputs 7-dim actions, while the world model was trained with up to 10-dim actions.
            a = torch.cat([a, a.new_zeros(3)], dim=-1)  # pad with zeros
            a = rescale_bridge_action(a)

            for i, x in wm.generate_chunk(a):
                new_frame = x[0, 0].cpu().numpy()
                new_frame = np.clip(new_frame * 255, 0, 255).astype(np.uint8)
                frames.append(new_frame)
        rollout_video = np.stack(frames)
        media.show_video(rollout_video, fps=20)
        avg_score = predict(rollout_video, task=task_i)
        scores.append(avg_score)
    return np.array(scores)

## Init world model

In [33]:
from world_model import WorldModel
import os

CHECKPOINTS_TO_KWARGS = {
    "bridge_v2_ckpt.pt": {  # The demo model checkpoint from our original arxiv release.
        "use_pixel_rope": True,
    },
    "200k_20frame_cfg_bridgev2_ckpt.pt": {  # New in-progress model with CFG and EMA.
        "use_pixel_rope": False,
        "default_cfg": 3.0,
    },
}
FILESERVER_URL = "https://85daf289d906.ngrok.app"  # This might change.

ckpt_path = "200k_20frame_cfg_bridgev2_ckpt.pt"  # Take your pick from above.
if not Path(ckpt_path).exists():
    ckpt_url = FILESERVER_URL + "/" + ckpt_path
    print(f"{ckpt_url=}")
    os.system(f"wget {ckpt_url}")

wm = WorldModel(ckpt_path, **CHECKPOINTS_TO_KWARGS[ckpt_path])

## Init OpenVLA

In [11]:
from transformers import AutoModelForVision2Seq, AutoProcessor

# Load Processor & VLA
processor = AutoProcessor.from_pretrained("openvla/openvla-7b", trust_remote_code=True)
vla = AutoModelForVision2Seq.from_pretrained(
    "openvla/openvla-7b",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
).cuda()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Expected `transformers==4.40.1` and `tokenizers==0.19.1` but got `transformers==4.53.0` and `tokenizers==0.21.4`; there might be inference-time regressions due to dependency changes. If in doubt, pleaseuse the above versions.
Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.62it/s]


## Scrape jpgs from the Bridge website

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def find_jpgs_http(url, session=None):
    if session is None:
        session = requests.Session()
    resp = session.get(url)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, 'html.parser')
    for link in soup.find_all('a'):
        href = link.get('href')
        if not href or href.startswith('?') or href.startswith('/'):
            continue

        full_url = urljoin(url, href)
        if href.endswith('/'):
            yield from find_jpgs_http(full_url, session=session)
        elif href.lower().endswith('im_0.jpg'):
            yield full_url

To choose a different task, edit the `base_url`, `instruction`, and `subtasks` below.

In [13]:
TASKS = {
    "https://rail.eecs.berkeley.edu/datasets/bridge_release/raw/bridge_data_v1/berkeley/toysink2_bww/put_carrot_on_plate/": {
        "instruction": "put carrot on plate",
        "subtasks": ["Pick up the carrot.", "Place the carrot on the plate."],
    },
    "https://rail.eecs.berkeley.edu/datasets/bridge_release/raw/bridge_data_v1/berkeley/toysink2_bww/put_eggplant_into_pot_or_pan/": {
        "instruction": "put eggplant into pot or pan",
        "subtasks": ["Pick up the eggplant.", "Place the eggplant into the pot or pan."],
    },
    "https://rail.eecs.berkeley.edu/datasets/bridge_release/raw/bridge_data_v2/deepthought_toykitchen1/close_microwave/": {
        "instruction": "close microwave",
        "subtasks": ["Push the microwave door.", "Close the microwave door completely."],
    },
    "https://rail.eecs.berkeley.edu/datasets/bridge_release/raw/bridge_data_v2/deepthought_toykitchen2/stack_blocks/": {
        "instruction": "stack blocks",
        "subtasks": ["Pick up a block.", "Place a block on top of another block."],
    },
    "https://rail.eecs.berkeley.edu/datasets/bridge_release/raw/bridge_data_v1/berkeley/toykitchen2/close_oven/": {
        "instruction": "close oven",
        "subtasks": ["Push the oven door.", "Close the oven door completely."],
    },
    "https://rail.eecs.berkeley.edu/datasets/bridge_release/raw/bridge_data_v2/deepthought_toykitchen1/open_microwave/": {
        "instruction": "open the microwave",
        "subtasks": ["Grab microwave handle.", "Pull microwave open."],
    },
    "https://rail.eecs.berkeley.edu/datasets/bridge_release/raw/bridge_data_v2/datacol2_folding_table_white_tray/sweep_granular/": {
        "instruction": "sweep up the beans",
        "subtasks": ["Grab the metal beam.", "Use the metal beam to sweep the beans."],
    },
    "https://rail.eecs.berkeley.edu/datasets/bridge_release/raw/bridge_data_v2/datacol1_toykitchen6/fold_cloth/": {
        "instruction": "fold cloth",
        "subtasks": ["Pick up one end of the cloth.", "Fold the cloth over."],
    },
}

In [34]:
import random
random.seed(0)
tasks = []
base_url = "https://rail.eecs.berkeley.edu/datasets/bridge_release/raw/bridge_data_v1/berkeley/toysink2_bww/put_carrot_on_plate/"
for jpg_url in find_jpgs_http(base_url):
    tasks.append(
        {
            "im_0_path": jpg_url,
            "instruction": TASKS[base_url]["instruction"],
            "subtasks": TASKS[base_url]["subtasks"],
        }
    )
tasks = random.sample(tasks, 20)

In [ ]:
scores = evaluate_policy(wm, vla, tasks)
print(f"Example task: {tasks[0]}")
print(f"Mean score: {np.mean(scores)=}")
print(f"STE: {np.std(scores) / len(scores)**0.5=}")